In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from collections import Counter

classes = ['Aorta', 'Flows', 'Other', 'V sign', 'X sign']

import os
import cv2
import numpy as np
from torch.utils.data import Dataset

class FetalEchoDataset(Dataset):
    def __init__(self, root_dir, phase='train', transform=None, selected_classes=None, x_sign_mode='both'):
        self.root_dir = root_dir
        self.phase = phase
        self.transform = transform

        all_classes = ['Aorta', 'Flows', 'Other', 'V sign', 'X sign']
        self.classes = selected_classes if selected_classes is not None else all_classes
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.samples = []

        if phase in ['train']:
            for cls in self.classes:
                if cls == 'X sign':
                    # Tùy chọn cách lấy ảnh của X sign
                    if x_sign_mode == 'clear':
                        subfolder = 'X sign/clear'
                        folder_path = os.path.join(root_dir, phase, subfolder)
                        if os.path.exists(folder_path):
                            for fname in os.listdir(folder_path):
                                if fname.endswith('.jpg') or fname.endswith('.png'):
                                    full_path = os.path.join(folder_path, fname)
                                    self.samples.append((full_path, self.class_to_idx['X sign']))
                    elif x_sign_mode == 'deformed':
                        subfolder = 'X sign/deformed'
                        folder_path = os.path.join(root_dir, phase, subfolder)
                        if os.path.exists(folder_path):
                            for fname in os.listdir(folder_path):
                                if fname.endswith('.jpg') or fname.endswith('.png'):
                                    full_path = os.path.join(folder_path, fname)
                                    self.samples.append((full_path, self.class_to_idx['X sign']))
                    elif x_sign_mode == 'both':
                        for subfolder in ['both']:
                            folder_path = os.path.join(root_dir, phase, 'X sign', subfolder)
                            if os.path.exists(folder_path):
                                for fname in os.listdir(folder_path):
                                    if fname.endswith('.jpg') or fname.endswith('.png'):
                                        full_path = os.path.join(folder_path, fname)
                                        self.samples.append((full_path, self.class_to_idx['X sign']))
                else:
                    folder_path = os.path.join(root_dir, phase, cls)
                    if os.path.exists(folder_path):
                        for fname in os.listdir(folder_path):
                            if fname.endswith('.jpg') or fname.endswith('.png'):
                                full_path = os.path.join(folder_path, fname)
                                self.samples.append((full_path, self.class_to_idx[cls]))
        elif phase in ['valid', 'test']:
            for cls in self.classes:
              folder_path = os.path.join(root_dir, phase, cls)
              if os.path.exists(folder_path):
                  for fname in os.listdir(folder_path):
                      if fname.endswith('.jpg') or fname.endswith('.png'):
                          full_path = os.path.join(folder_path, fname)
                          self.samples.append((full_path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, label

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
main_folder = '/content/drive/MyDrive/Colab Notebooks/dataset/processedfetal_delation'
# Set image size if needed (optional)
IMG_SIZE = (224, 224)

# Data augmentations for training set
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip ngang
    transforms.RandomApply([
        transforms.RandomRotation(degrees=10)  # Xoay ±10 độ
    ], p=0.75),
    transforms.RandomResizedCrop(size=IMG_SIZE, scale=(1.0, 1.1)),
    # Phóng to trong khoảng 1.0–1.1x và resize về IMG_SIZE
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Thay đổi sáng/tương phản
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),  # Warp đối xứng
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Chuẩn hóa ảnh màu RGB
])
# For validation/test: no augmentation
val_test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Chuẩn hóa ảnh màu RGB
])

# Chỉ dùng X sign rõ cấu trúc
train_dataset_clear = FetalEchoDataset(main_folder, phase='train', transform=train_transforms, x_sign_mode='clear')

# Hoặc chỉ dùng X sign biến dạng
train_dataset_deformed = FetalEchoDataset(main_folder, phase='train', transform=train_transforms, x_sign_mode='deformed')

# Hoặc dùng cả hai (mặc định)
train_dataset_both = FetalEchoDataset(main_folder, phase='train', transform=train_transforms, x_sign_mode='both')


# Load datasets
valid_dataset = FetalEchoDataset(main_folder, phase='valid', transform=val_test_transforms)
test_dataset  = FetalEchoDataset(main_folder, phase='test',  transform=val_test_transforms)

# DataLoaders
BATCH_SIZE = 32
train_dataset_both_loader = DataLoader(train_dataset_both, batch_size=BATCH_SIZE, shuffle=True)
train_dataset_deformed_loader =DataLoader(train_dataset_deformed, batch_size=BATCH_SIZE, shuffle=True)
train_dataset_clear_loader = DataLoader(train_dataset_clear, batch_size=BATCH_SIZE, shuffle=True)

valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# CBAM
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x))

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.ca = ChannelAttention(channels, reduction)
        self.sa = SpatialAttention()

    def forward(self, x):
        return self.sa(self.ca(x) * x) * (self.ca(x) * x)

# Model
from torchvision.models import DenseNet201_Weights
class DenseNet201_CBAM(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        base = models.densenet201(weights=DenseNet201_Weights.DEFAULT)
        self.features = base.features
        self.cbam1 = CBAM(256)
        self.cbam2 = CBAM(1792)
        self.cbam3 = CBAM(1920)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1920, num_classes)

    def forward(self, x):
        x = self.features.conv0(x)
        x = self.features.norm0(x)
        x = self.features.relu0(x)
        x = self.features.pool0(x)

        x = self.features.denseblock1(x)
        x = self.cbam1(x)
        x = self.features.transition1(x)

        x = self.features.denseblock2(x)
        x = self.features.transition2(x)

        x = self.features.denseblock3(x)
        x = self.cbam2(x)
        x = self.features.transition3(x)

        x = self.features.denseblock4(x)
        x = self.features.norm5(x)
        x = self.cbam3(x)

        x = self.pool(x).view(x.size(0), -1)
        return self.fc(x)


In [ ]:
del

In [ ]:
# Focal Loss with class weights
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            at = self.alpha[targets]
            loss = at * (1 - pt) ** self.gamma * ce_loss
        else:
            loss = (1 - pt) ** self.gamma * ce_loss
        return loss.mean()
# Device & training
best_f1 = 0.0
pathbestmodel = ''
from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet201_CBAM(num_classes=len(classes)).to(device)
alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_losses, valid_losses, accuracies = [], [], []
num_epochs = 10
count=0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataset_clear_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset_clear_loader.dataset)
    train_losses.append(epoch_loss)

    model.eval()
    valid_loss, correct = 0.0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_epoch_loss = valid_loss / len(valid_loader.dataset)
    accuracy = correct.double() / len(valid_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    valid_losses.append(valid_epoch_loss)
    accuracies.append(accuracy.item())

    print(f"Phase 1: Epoch {epoch+1}: Train Loss = {epoch_loss:.4f}, Val Loss = {valid_epoch_loss:.4f}, Acc = {accuracy:.4f}, F1 Score(macro) = {f1:.4f} ")
    if f1 > best_f1:
        best_f1 = f1
        pathbestmodel = f'/content/drive/MyDrive/Colab Notebooks/model/model_test/Densenet_CBAM_with_Xsign_clear{count}.pt'
        torch.save(model.state_dict(), pathbestmodel)
        count += 1
        print("✅ Saved best model!")

Phase 1: Epoch 1: Train Loss = 0.4470, Val Loss = 0.3308, Acc = 0.9135, F1 Score(macro) = 0.8723 
✅ Saved best model!
Phase 1: Epoch 2: Train Loss = 0.0879, Val Loss = 0.2821, Acc = 0.9260, F1 Score(macro) = 0.8683 
Phase 1: Epoch 3: Train Loss = 0.0464, Val Loss = 0.2740, Acc = 0.9268, F1 Score(macro) = 0.8629 
Phase 1: Epoch 4: Train Loss = 0.0282, Val Loss = 0.4139, Acc = 0.8979, F1 Score(macro) = 0.8079 
Phase 1: Epoch 5: Train Loss = 0.0322, Val Loss = 0.2698, Acc = 0.9253, F1 Score(macro) = 0.8734 
✅ Saved best model!
Phase 1: Epoch 6: Train Loss = 0.0194, Val Loss = 0.2337, Acc = 0.9290, F1 Score(macro) = 0.8664 
Phase 1: Epoch 7: Train Loss = 0.0234, Val Loss = 0.2824, Acc = 0.9534, F1 Score(macro) = 0.9026 
✅ Saved best model!
Phase 1: Epoch 8: Train Loss = 0.0226, Val Loss = 0.2058, Acc = 0.9445, F1 Score(macro) = 0.9114 
✅ Saved best model!
Phase 1: Epoch 9: Train Loss = 0.0070, Val Loss = 0.2636, Acc = 0.9445, F1 Score(macro) = 0.8967 
Phase 1: Epoch 10: Train Loss = 0.0075

In [ ]:
# Focal Loss with class weights
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            at = self.alpha[targets]
            loss = at * (1 - pt) ** self.gamma * ce_loss
        else:
            loss = (1 - pt) ** self.gamma * ce_loss
        return loss.mean()

In [ ]:

# Device & training
best_f1 = 0.9114
from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet201_CBAM(num_classes=len(classes)).to(device)
alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 184MB/s]


In [ ]:
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, T=2.0):
    student_soft = F.log_softmax(student_logits / T, dim=1)
    teacher_soft = F.softmax(teacher_logits / T, dim=1)
    loss = F.kl_div(student_soft, teacher_soft, reduction='batchmean') * (T * T)
    return loss

In [ ]:
import copy

In [ ]:
pathbestmodel = '/content/drive/MyDrive/Colab Notebooks/model/model_test/Densenet_CBAM_with_Xsign_clear3.pt'

In [ ]:
model.load_state_dict(torch.load(pathbestmodel))
model.eval()
# lưu model cũ
pathbestmodel_old = pathbestmodel
# Load lại mô hình phase 1 làm old_model (nếu chưa có)
old_model = copy.deepcopy(model)
old_model.eval()
for param in old_model.parameters():
    param.requires_grad = False

In [ ]:
#phase 2 version 3 thay đổi bot -> deformed loss = 40 mới +60 cũ
lambda_distill = 0.6
num_epochs = 10
count = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []
for epoch in range(num_epochs):
    model.train()
    running_loss, correct_train, total_train = 0.0, 0, 0

    for inputs, labels in train_dataset_deformed_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        with torch.no_grad():
            teacher_outputs = old_model(inputs)

        loss_focal = criterion(outputs, labels)
        loss_distill = distillation_loss(outputs, teacher_outputs, T=2.0)
        loss = loss_focal + lambda_distill * loss_distill

        loss.backward()

        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    epoch_loss = running_loss / len(train_dataset_deformed_loader.dataset)
    train_acc = correct_train / total_train

    train_losses.append(epoch_loss)
    train_accuracies.append(train_acc)


    # Đánh giá
    model.eval()
    valid_loss, correct = 0.0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_epoch_loss = valid_loss / len(valid_loader.dataset)
    accuracy = correct.double() / len(valid_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    valid_losses.append(valid_epoch_loss)
    valid_accuracies.append(accuracy.item())

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {epoch_loss:.4f}, Train Acc = {train_acc:.4f}, "
          f"Val Loss = {valid_epoch_loss:.4f}, Val Acc = {accuracy:.4f}, "
          f"F1 Score (macro) = {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        pathbestmodel = f'/content/drive/MyDrive/Colab Notebooks/model/model_test/Densenet_CBAM_with_Xsign_deformed{count}.pt'
        torch.save(model.state_dict(), pathbestmodel)
        count += 1
        print("✅ Saved best model!")


Epoch 1: Train Loss = 0.0878, Train Acc = 0.9914, Val Loss = 0.1538, Val Acc = 0.9467, F1 Score (macro) = 0.9159
✅ Saved best model!
Epoch 2: Train Loss = 0.0974, Train Acc = 0.9904, Val Loss = 0.1860, Val Acc = 0.9475, F1 Score (macro) = 0.9123
Epoch 3: Train Loss = 0.0650, Train Acc = 0.9940, Val Loss = 0.1588, Val Acc = 0.9534, F1 Score (macro) = 0.9130
Epoch 4: Train Loss = 0.0553, Train Acc = 0.9958, Val Loss = 0.1579, Val Acc = 0.9593, F1 Score (macro) = 0.9287
✅ Saved best model!
Epoch 5: Train Loss = 0.0457, Train Acc = 0.9969, Val Loss = 0.1698, Val Acc = 0.9519, F1 Score (macro) = 0.9182
Epoch 6: Train Loss = 0.0390, Train Acc = 0.9982, Val Loss = 0.1630, Val Acc = 0.9549, F1 Score (macro) = 0.9263
Epoch 7: Train Loss = 0.0456, Train Acc = 0.9977, Val Loss = 0.1849, Val Acc = 0.9578, F1 Score (macro) = 0.9243
Epoch 8: Train Loss = 0.0676, Train Acc = 0.9927, Val Loss = 0.1871, Val Acc = 0.9593, F1 Score (macro) = 0.9254
Epoch 9: Train Loss = 0.0498, Train Acc = 0.9956, Val Lo

In [ ]:
del model
del old_model

In [ ]:

from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet201_CBAM(num_classes=len(classes)).to(device)
alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
model.load_state_dict(torch.load(pathbestmodel))
model.eval()
# lưu model cũ
pathbestmodel_old = pathbestmodel
# Load lại mô hình phase 1 làm old_model (nếu chưa có)
old_model = copy.deepcopy(model)
old_model.eval()
for param in old_model.parameters():
    param.requires_grad = False

In [ ]:
#phase 3
lambda_distill = 0.6
num_epochs = 10
count = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []
for epoch in range(num_epochs):
    model.train()
    running_loss, correct_train, total_train = 0.0, 0, 0

    for inputs, labels in train_dataset_both_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        with torch.no_grad():
            teacher_outputs = old_model(inputs)

        loss_focal = criterion(outputs, labels)
        loss_distill = distillation_loss(outputs, teacher_outputs, T=2.0)
        loss = loss_focal + lambda_distill * loss_distill

        loss.backward()

        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    epoch_loss = running_loss / len(train_dataset_both_loader.dataset)
    train_acc = correct_train / total_train

    train_losses.append(epoch_loss)
    train_accuracies.append(train_acc)


    # Đánh giá
    model.eval()
    valid_loss, correct = 0.0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_epoch_loss = valid_loss / len(valid_loader.dataset)
    accuracy = correct.double() / len(valid_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    valid_losses.append(valid_epoch_loss)
    valid_accuracies.append(accuracy.item())

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {epoch_loss:.4f}, Train Acc = {train_acc:.4f}, "
          f"Val Loss = {valid_epoch_loss:.4f}, Val Acc = {accuracy:.4f}, "
          f"F1 Score (macro) = {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        pathbestmodel = f'/content/drive/MyDrive/Colab Notebooks/model/model_test/Densenet_CBAM_with_Xsign_both{count}.pt'
        torch.save(model.state_dict(), pathbestmodel)
        count += 1
        print("✅ Saved best model!")


Epoch 1: Train Loss = 0.0570, Train Acc = 0.9925, Val Loss = 0.1605, Val Acc = 0.9556, F1 Score (macro) = 0.9229
Epoch 2: Train Loss = 0.0460, Train Acc = 0.9947, Val Loss = 0.1710, Val Acc = 0.9593, F1 Score (macro) = 0.9319
✅ Saved best model!
Epoch 3: Train Loss = 0.0440, Train Acc = 0.9960, Val Loss = 0.1764, Val Acc = 0.9527, F1 Score (macro) = 0.9173
Epoch 4: Train Loss = 0.0302, Train Acc = 0.9977, Val Loss = 0.1649, Val Acc = 0.9541, F1 Score (macro) = 0.9239
Epoch 5: Train Loss = 0.0302, Train Acc = 0.9980, Val Loss = 0.1856, Val Acc = 0.9512, F1 Score (macro) = 0.9155
Epoch 6: Train Loss = 0.0277, Train Acc = 0.9972, Val Loss = 0.1732, Val Acc = 0.9556, F1 Score (macro) = 0.9237
Epoch 7: Train Loss = 0.0210, Train Acc = 0.9995, Val Loss = 0.1719, Val Acc = 0.9549, F1 Score (macro) = 0.9223
Epoch 8: Train Loss = 0.0231, Train Acc = 0.9982, Val Loss = 0.1586, Val Acc = 0.9534, F1 Score (macro) = 0.9195
Epoch 9: Train Loss = 0.0708, Train Acc = 0.9895, Val Loss = 0.2266, Val Acc

In [ ]:
#phase 3
lambda_distill = 0.6
num_epochs = 10
count = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alpha_weights = torch.tensor([1.2, 1.0, 2.5, 1.2, 3.0], dtype=torch.float32).to(device)
criterion = FocalLoss(alpha=alpha_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []
for epoch in range(num_epochs):
    model.train()
    running_loss, correct_train, total_train = 0.0, 0, 0

    for inputs, labels in train_dataset_both_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        with torch.no_grad():
            teacher_outputs = old_model(inputs)

        loss_focal = criterion(outputs, labels)
        loss_distill = distillation_loss(outputs, teacher_outputs, T=2.0)
        loss = loss_focal + lambda_distill * loss_distill

        loss.backward()

        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    epoch_loss = running_loss / len(train_dataset_both_loader.dataset)
    train_acc = correct_train / total_train

    train_losses.append(epoch_loss)
    train_accuracies.append(train_acc)


    # Đánh giá
    model.eval()
    valid_loss, correct = 0.0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_epoch_loss = valid_loss / len(valid_loader.dataset)
    accuracy = correct.double() / len(valid_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    valid_losses.append(valid_epoch_loss)
    valid_accuracies.append(accuracy.item())

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {epoch_loss:.4f}, Train Acc = {train_acc:.4f}, "
          f"Val Loss = {valid_epoch_loss:.4f}, Val Acc = {accuracy:.4f}, "
          f"F1 Score (macro) = {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        pathbestmodel = f'/content/drive/MyDrive/Colab Notebooks/model/model_test/Densenet_CBAM_with_Xsign_both{count}.pt'
        torch.save(model.state_dict(), pathbestmodel)
        count += 1
        print("✅ Saved best model!")


Epoch 1: Train Loss = 0.0406, Train Acc = 0.9960, Val Loss = 0.1800, Val Acc = 0.9593, F1 Score (macro) = 0.9257
Epoch 2: Train Loss = 0.0281, Train Acc = 0.9992, Val Loss = 0.1551, Val Acc = 0.9601, F1 Score (macro) = 0.9333
✅ Saved best model!
Epoch 3: Train Loss = 0.0202, Train Acc = 0.9992, Val Loss = 0.1492, Val Acc = 0.9578, F1 Score (macro) = 0.9299
Epoch 4: Train Loss = 0.0368, Train Acc = 0.9960, Val Loss = 0.2034, Val Acc = 0.9578, F1 Score (macro) = 0.9313
Epoch 5: Train Loss = 0.0336, Train Acc = 0.9972, Val Loss = 0.2338, Val Acc = 0.9534, F1 Score (macro) = 0.9215
Epoch 6: Train Loss = 0.0337, Train Acc = 0.9965, Val Loss = 0.2082, Val Acc = 0.9519, F1 Score (macro) = 0.9176
Epoch 7: Train Loss = 0.0218, Train Acc = 0.9982, Val Loss = 0.1374, Val Acc = 0.9667, F1 Score (macro) = 0.9430
✅ Saved best model!
Epoch 8: Train Loss = 0.0166, Train Acc = 0.9992, Val Loss = 0.1668, Val Acc = 0.9601, F1 Score (macro) = 0.9300
Epoch 9: Train Loss = 0.0153, Train Acc = 0.9997, Val Lo

In [2]:
!nvidia-smi

Fri Jul 11 08:36:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----